## Chess.com API project for extracting data for any user in a CSV formate.

In [1]:
!pip install pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install chess.com

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


In [4]:
from chessdotcom import get_leaderboards, get_player_stats, get_player_game_archives 
import pprint
import requests
import re

Below code is to extract a Dictonery where a key is the ECO code (which is different for every opening) and the values contain the opening name and inital moves for every coressponding ECO codes. 

In [27]:
#ECO codes - chess opning generator 
# Open the text file for reading
def split_(line):
  split_line = line.strip().split('\t')
  return split_line

with open('chess_openings.txt', 'r') as file:

    opening_dict = {}
    # Read each line in the file
    a=0
    for line in file:
      if a==0:
        ret = split_(line)
        eco_code = ret[0]
        opening_name = ret[1]
        a=1
      else:
        ret = split_(line)
        mo=ret[0]
        a=0
        opening_dict[eco_code] = (opening_name, mo)
    


In [6]:
#function to retrive data from each game;
#input - a Game
#list containg the following:
#[1.mode of game; 2.user played with; 3.Game result; 4.raiting of the user; 5.rating of opponent; 6.opening played by user; 7.Inital moves; 8.Url of the game]

def get_data(in_game,user_name):
  return_list = []
  info_game=in_game
  tocken = 1

  aval_info = info_game.keys()
  req_info = ['url', 'pgn', 'end_time', 'rated', 'tcn', 'uuid', 'initial_setup', 'fen', 'time_class', 'rules', 'white', 'black']
  for i in req_info:
    if i in aval_info:
      pass
    else:
      tocken = 0
      break
  
  if tocken ==1:
    game_str = info_game['pgn']
    mat = re.search(r'\[ECO "(.*?)"\]', game_str)
    if mat is None:
      tocken = 0
  else:
    pass
  if info_game['rules'] == 'chess' and tocken ==1:
    #start

    #Date of the game played:
    game_str = info_game['pgn']
    match = re.search(r'\[Date "(\d{4}\.\d{2}\.\d{2})"\]', game_str)
    if match:
      date = match.group(1)
      return_list.append(date)


    #For the Mode of the chesss game,
    game_mode = info_game['time_class']
    return_list.append(game_mode)

    #For fingind the user played with white pices or black,
    if info_game['white']['username'] == user_name:
      played_with = "white"
      opp_played_with = "black"
    else:
      played_with = "black"
      opp_played_with = "white"
    return_list.append(played_with)

    #win or Loss for the user in the game:
    game_str = info_game['pgn']
    game_result = re.search(r'\[Result "(.*?)"\]', game_str).group(1)
    if game_result == "1/2-1/2":
      user_result = 'draw'
    elif played_with == "white":
      if game_result == "1-0":
        user_result = "win"
      else:
        user_result = "loss"
    else:
      if game_result == "0-1":
        user_result = "win"
      else:
        user_result = "loss"
    return_list.append(user_result)

    #rating of the user and opponent in the game:
    user_rating = info_game[played_with]['rating']
    opp_rating = info_game[opp_played_with]['rating']
    return_list.append(user_rating)
    return_list.append(opp_rating)

  
    #opening played by user- opening Name and inital moves:
    game_str = info_game['pgn']
    eco_code = re.search(r'\[ECO "(.*?)"\]', game_str).group(1)
    played = opening_dict[eco_code]
    opening_name = played[0]
    opening_moves = played[1]
    return_list.append(opening_name)
    return_list.append(opening_moves)

    #all the moves:
    pgn = '[Event "Live Chess"]\n' # the PGN notation from the dictionary
    moves = re.findall('\d+\.\s(\S+)', game_str) # extract all the moves using a regex
    return_list.append(moves)
    

    #url of the game:
    url_ofgame = info_game['url']
    return_list.append(url_ofgame)
    
  else:
    pass
  return return_list


In [8]:
def get_thedata(username):
  data = get_player_game_archives(username).json
  all_games = data['archives']
  return all_games

In the below code enter the username of the player.

In [25]:
#Enter the chess.com username of the player you want the data off:

user_name = "GothamChess"


In [26]:
#Creating a dataframe for all the games played by the user:

import pandas as pd

df_list=[]

def get_thedata(username):
  data = get_player_game_archives(username).json
  all_games = data['archives']
  return all_games

all_games = get_thedata(user_name)
req_game_months = all_games[0:]


for i in req_game_months:
  url = i
  games = requests.get(url).json()
  for game_no in games['games']:
    ret_list = get_data(game_no,user_name)
    df_list.append(ret_list)

df = pd.DataFrame(df_list, columns=["game_date","game_mode","player_color","game_result","player_rating","opponent_rating","opening_name","opening_moves","all_moves","game_url"])

#Converting the dataframe to csv file using the function defined above:
df.to_csv(str(user_name)+".csv")


In [28]:
df.head()

,game_date,game_mode,player_color,game_result,player_rating,opponent_rating,opening_name,opening_moves,all_moves,game_url
0,2017.03.14,blitz,white,win,1917.0,1635.0,Queen's Pawn Game,1 d4,"[d4, Nf3, c4, Nc3, Bf4, e3, Ne5, g4, gxf5, Rg1...",https://www.chess.com/game/live/1996257721
1,2017.03.14,blitz,black,win,2021.0,1804.0,Reti Opening,1 Nf3,"[Nf3, g3, Bg2, Ne5, b3, Bb2, Nc4, Bxg7, c3, bx...",https://www.chess.com/game/live/1996259836
2,2017.03.14,blitz,white,win,2098.0,1876.0,Queen's Pawn Game (with ...d6),1 d4 d6,"[d4, c4, Nc3, e4, Be2, g4, d5, h4, h5, hxg6, g...",https://www.chess.com/game/live/1996265351
3,2017.03.14,blitz,black,win,2230.0,2267.0,Queen's Gambit Accepted,1 d4 d5 2 c4 dxc4 3 Nf3 Nf6 4 Nc3,"[d4, c4, Nf3, Nc3, e4, e5, a4, axb5, bxa6, Rxa...",https://www.chess.com/game/live/1996269835
4,2017.03.14,blitz,white,win,2311.0,2234.0,Queen's Pawn Game,1 d4 Nf6,"[d4, Bg5, Qd3, e4, Nd2, h4, Be3, f3, g4, Ne2, ...",https://www.chess.com/game/live/1996275562
